## Context Retrieval


In [ ]:
train_challenge = pd.read_csv('/content/ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Train.csv', )
train_challenge.head(3)

,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,schoolGrade,year,question,subject,category
0,Mercury_SC_415702,415702,1,A,1,0,Mercury,3,2015,George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat? (A) dry palms (B) wet palms (C) palms covered with oil (D) palms covered with lotion,NaN,Train
1,MCAS_2009_5_6516,6516,1,B,1,0,MCAS,5,2009,Which of the following statements best explains why magnets usually stick to a refrigerator door? (A) The refrigerator door is smooth. (B) The refrigerator door contains iron. (C) The refrigerator door is a good conductor. (D) The refrigerator door has electric wires in it.,NaN,Train
2,Mercury_7233695,7233695,1,B,1,0,Mercury,9,2015,A fold observed in layers of sedimentary rock most likely resulted from the (A) cooling of flowing magma. (B) converging of crustal plates. (C) deposition of river sediments. (D) solution of carbonate minerals.,NaN,Train


In [ ]:
dev_challenge = pd.read_csv('/content/ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Dev.csv' )
test_challenge = pd.read_csv('/content/ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Test.csv')

In [ ]:
train_easy = pd.read_csv('/content/ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Train.csv' )
dev_easy = pd.read_csv('/content/ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Dev.csv' )
test_easy = pd.read_csv('/content/ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Test.csv')

In [ ]:
import itertools

def question_answer_extraction(data):

    question, answer = [], []

    for ques_ans in data:
        ques_ans = ques_ans.split('(')
        ques = ques_ans[0]
        question.append(ques)

        ans = [re.findall(r'\)(.*)', ans) for ans in ques_ans[1:]]
        ans = list(itertools.chain(*ans))
        answer.append(ans)

    return question , answer

In [ ]:
train_challenge['only_questions'], train_challenge['only_answers']  = question_answer_extraction(train_challenge['question'])
dev_challenge['only_questions'], dev_challenge['only_answers']  = question_answer_extraction(dev_challenge['question'])
test_challenge['only_questions'], test_challenge['only_answers']  = question_answer_extraction(test_challenge['question'])

train_easy['only_questions'], train_easy['only_answers']  = question_answer_extraction(train_easy['question'])
dev_easy['only_questions'], dev_easy['only_answers']  = question_answer_extraction(dev_easy['question'])
test_easy['only_questions'], test_easy['only_answers']  = question_answer_extraction(test_easy['question'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def top_50_context_extraction(data):
    '''
    function to get relevant context from ARC and ARISTO corpus
    '''

    question= data['only_questions']
    answer = data['only_answers']

    list_of_context= []
    list_of_score =[]
    scaler = MinMaxScaler()
    #iterate through each question
    for i, que in enumerate(question):
        context,score = [],[]
        #iterate through each option of corresponding question
        for ans in answer[i]:
            # body of searching or querying relevant text
            query ={
            "size":40, # it will get 40 text sentences
            "query":{
                "bool":{
                    "must":[# it must search with question plus answer text
                            {"match":{"sentence": str(que) + ' ' + str(ans)}}
                    ],
                    "should":[# atleast it should search answer related text
                            {"match":{"sentence": str(ans)}}
                    ]}}}

            response_arc = es.search(index='corpus_arc', body=query)
            response_aristo = es.search(index='corpus_aristo', body=query)

            top_documents = {}
            # search document from ARC corpus
            # will save both search related text in a single list
            for hit in response_arc['hits']['hits']:
                top_documents[hit['_source']['sentence']] = hit['_score']
            # search document from Aristo corpus
            for hit in response_aristo['hits']['hits']:
                top_documents[hit['_source']['sentence']] = hit['_score']

            # now we will sort text document in descending order
            # will only keep top 50 document text as per their scores
            top_document = sorted(top_documents.items(), key=lambda item: item[1], reverse=True)
            context.append(list(top_documents.keys())[:50])

            values= scaler.fit_transform(np.array(list(top_documents.values())[:50]).reshape(-1, 1))
            score.append(list(chain(*values)))
        list_of_context.append(context)
        list_of_score.append(score)

    return list_of_context, list_of_score

In [ ]:
train_challenge['context'], train_challenge['score']= top_50_context_extraction(train_challenge)
dev_challenge['context'], dev_challenge['score']= top_50_context_extraction(dev_challenge)
test_challenge['context'], test_challenge['score']= top_50_context_extraction(test_challenge)

train_easy['context'], train_easy['score']= top_50_context_extraction(train_easy)
dev_easy['context'], dev_easy['score']= top_50_context_extraction(dev_easy)
test_easy['context'], test_easy['score']= top_50_context_extraction(test_easy)

In [ ]:
pickle.dump((train_challenge, dev_challenge, test_challenge, train_easy, dev_easy, test_easy),
            open('/content/drive/MyDrive/my assignments/33. A12 Reasoning Challenge- Self case study 2/arc_datasets.pkl','wb'))

In [ ]:
train_challenge, dev_challenge, test_challenge, train_easy, dev_easy, test_easy= \
pickle.load(open('/content/drive/MyDrive/my assignments/33. A12 Reasoning Challenge- Self case study 2/arc_datasets.pkl', 'rb'))